# Bandits - part 2

## Reward distribution

We are now going to vary the reward distributions and investigate whether the experimental results we had previously when the true Q-values are in $\mathcal{N}(0, 1)$ and the rewards have a variance of 1 still hold.

**Q:** Let's now change the distribution of true Q-values from $\mathcal{N}(0, 1)$ to $\mathcal{N}(10, 10)$ and re-run the algorithms. What happens and why? Modify the values of `epsilon` and `tau` to try to get a better behavior.

In [ ]:
# Learning rate
alpha = 0.1

# Exploration
epsilon = 0.1
tau = 3.0

# Setup
nb_trials = 200
nb_steps = 1000

# Store all received rewards and the optimality in a big matrix
rewards_greedy = np.zeros((nb_trials, nb_steps))
optimal_greedy = np.zeros((nb_trials, nb_steps))

rewards_egreedy = np.zeros((nb_trials, nb_steps))
optimal_egreedy = np.zeros((nb_trials, nb_steps))

rewards_softmax = np.zeros((nb_trials, nb_steps))
optimal_softmax = np.zeros((nb_trials, nb_steps))

# 200 runs
for trial in range(nb_trials):
    
    # True Q-values
    Q_star = rng.normal(10, 10, nb_actions)
    a_star = rng.choice(np.where(Q_star == Q_star.max())[0])
    
    # Estimates
    Q_greedy = np.zeros(nb_actions)
    Q_egreedy = np.zeros(nb_actions)
    Q_softmax = np.zeros(nb_actions)
    
    # For 1000 plays
    for step in range(nb_steps):    
        
        # greedy
        a = rng.choice(np.where(Q_greedy == Q_greedy.max())[0])
        r = get_reward(Q_star, a)
        rewards_greedy[trial, step] = r
        if a == a_star:
            optimal_greedy[trial, step] = 1.0 
        Q_greedy[a] += alpha * (r - Q_greedy[a])
        
        # epsilon-greedy
        a = rng.choice(np.where(Q_egreedy == Q_egreedy.max())[0])
        if np.random.rand() < epsilon:
            a = np.random.choice(actions[actions !=a])
        r = get_reward(Q_star, a)
        rewards_egreedy[trial, step] = r
        if a == a_star:
            optimal_egreedy[trial, step] = 1.0 
        Q_egreedy[a] += alpha * (r - Q_egreedy[a])
        
        # Softmax            
        logit = np.exp((Q_softmax - Q_softmax.max())/tau)
        proba_softmax = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_softmax) 
        r = get_reward(Q_star, a)
        rewards_softmax[trial, step] = r
        if a == a_star:
            optimal_softmax[trial, step] = 1.0            
        Q_softmax[a] += alpha * (r - Q_softmax[a])
    
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.mean(rewards_greedy, axis=0), label="greedy")
plt.plot(np.mean(rewards_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(rewards_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Mean Reward")
plt.legend()
plt.subplot(122)
plt.plot(np.mean(100*optimal_greedy, axis=0), label="greedy")
plt.plot(np.mean(100*optimal_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(100*optimal_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Optimal action (%)")
plt.ylim((0,100))
plt.legend()
plt.show()

**A:** Greedy does not work anymore and stays at chance level. The first action it samples will probably have a non-zero reward, so its estimated Q-value becomes positive (initial estimate of 0) and it will stay the greedy action all along.

$\epsilon$-greedy still works quite well (perhaps a bit slower as the estimates must go from 0 to 10), even with its default value of 0.1.

Softmax does not work unless you increase the temperature to 3 or so. The correct value of `tau` depends on the scaling of the Q-values, so it has to be adapted to every new problem, contrary to $\epsilon$-greedy. But with the correct value of `tau`, you get a good solution much earlier.

## Optimistic initialization

The initial estimates of 0 are now very **pessimistic** compared to the average reward you can get (10). This was not the case in the original setup.

**Q:** Let's change the initial value of the estimates to 10 for each action for each algorithm. What happens? Conclude on the importance of reward scaling.

In [ ]:
# Learning rate
alpha = 0.1

# Exploration
epsilon = 0.1
tau = 3.0

# Setup
nb_trials = 200
nb_steps = 1000

# Store all received rewards and the optimality in a big matrix
rewards_greedy = np.zeros((nb_trials, nb_steps))
optimal_greedy = np.zeros((nb_trials, nb_steps))

rewards_egreedy = np.zeros((nb_trials, nb_steps))
optimal_egreedy = np.zeros((nb_trials, nb_steps))

rewards_softmax = np.zeros((nb_trials, nb_steps))
optimal_softmax = np.zeros((nb_trials, nb_steps))

# 200 runs
for trial in range(nb_trials):
    
    # True Q-values
    Q_star = rng.normal(10, 10, nb_actions)
    a_star = rng.choice(np.where(Q_star == Q_star.max())[0])
    
    # Estimates
    Q_greedy = 10*np.ones(nb_actions)
    Q_egreedy = 10*np.ones(nb_actions)
    Q_softmax = 10*np.ones(nb_actions)
    
    # For 1000 plays
    for step in range(nb_steps):    
        
        # greedy
        a = rng.choice(np.where(Q_greedy == Q_greedy.max())[0])
        r = get_reward(Q_star, a)
        rewards_greedy[trial, step] = r
        if a == a_star:
            optimal_greedy[trial, step] = 1.0 
        Q_greedy[a] += alpha * (r - Q_greedy[a])
        
        # epsilon-greedy
        a = rng.choice(np.where(Q_egreedy == Q_egreedy.max())[0])
        if rng.random() < epsilon:
            a = rng.choice(actions[actions !=a])
        r = get_reward(Q_star, a)
        rewards_egreedy[trial, step] = r
        if a == a_star:
            optimal_egreedy[trial, step] = 1.0 
        Q_egreedy[a] += alpha * (r - Q_egreedy[a])
        
        # Softmax            
        logit = np.exp((Q_softmax - Q_softmax.max())/tau)
        proba_softmax = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_softmax) 
        r = get_reward(Q_star, a)
        rewards_softmax[trial, step] = r
        if a == a_star:
            optimal_softmax[trial, step] = 1.0            
        Q_softmax[a] += alpha * (r - Q_softmax[a])
    
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.mean(rewards_greedy, axis=0), label="greedy")
plt.plot(np.mean(rewards_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(rewards_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Mean Reward")
plt.legend()
plt.subplot(122)
plt.plot(np.mean(100*optimal_greedy, axis=0), label="greedy")
plt.plot(np.mean(100*optimal_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(100*optimal_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Optimal action (%)")
plt.ylim((0,100))
plt.legend()
plt.show()

**A:** Now we are back to quite the same results as before (greedy might still be worse, but not at chance level anymore). This shows the importance of **reward scaling**: the amplitude of the rewards influences a lot the success of the different methods ($\epsilon$-greedy is more robust). This mean you need to know the mean expected reward in advance, but you are not supposed to know that as you have not sampled anything at the beginning...

Let's now use **optimistic initialization**, i.e. initialize the estimates to a much higher value than what is realistic.

**Q:** Implement optimistic initialization by initializing the estimates of all three algorithms to 20. What happens?

In [ ]:
# Learning rate
alpha = 0.1

# Exploration
epsilon = 0.1
tau = 3.0

# Setup
nb_trials = 200
nb_steps = 1000

# Store all received rewards and the optimality in a big matrix
rewards_greedy = np.zeros((nb_trials, nb_steps))
optimal_greedy = np.zeros((nb_trials, nb_steps))

rewards_egreedy = np.zeros((nb_trials, nb_steps))
optimal_egreedy = np.zeros((nb_trials, nb_steps))

rewards_softmax = np.zeros((nb_trials, nb_steps))
optimal_softmax = np.zeros((nb_trials, nb_steps))

# 200 runs
for trial in range(nb_trials):
    
    # True Q-values
    Q_star = rng.normal(10, 10, nb_actions)
    a_star = rng.choice(np.where(Q_star == Q_star.max())[0])
    
    # Estimates
    Q_greedy = 20*np.ones(nb_actions)
    Q_egreedy = 20*np.ones(nb_actions)
    Q_softmax = 20*np.ones(nb_actions)
    
    # For 1000 plays
    for step in range(nb_steps):    
        
        # greedy
        a = rng.choice(np.where(Q_greedy == Q_greedy.max())[0])
        r = get_reward(Q_star, a)
        rewards_greedy[trial, step] = r
        if a == a_star:
            optimal_greedy[trial, step] = 1.0 
        Q_greedy[a] += alpha * (r - Q_greedy[a])
        
        # epsilon-greedy
        a = rng.choice(np.where(Q_egreedy == Q_egreedy.max())[0])
        if rng.random() < epsilon:
            a = rng.choice(actions[actions !=a])
        r = get_reward(Q_star, a)
        rewards_egreedy[trial, step] = r
        if a == a_star:
            optimal_egreedy[trial, step] = 1.0 
        Q_egreedy[a] += alpha * (r - Q_egreedy[a])
        
        # Softmax            
        logit = np.exp((Q_softmax - Q_softmax.max())/tau)
        proba_softmax = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_softmax) 
        r = get_reward(Q_star, a)
        rewards_softmax[trial, step] = r
        if a == a_star:
            optimal_softmax[trial, step] = 1.0            
        Q_softmax[a] += alpha * (r - Q_softmax[a])
    
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.mean(rewards_greedy, axis=0), label="greedy")
plt.plot(np.mean(rewards_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(rewards_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Mean Reward")
plt.legend()
plt.subplot(122)
plt.plot(np.mean(100*optimal_greedy, axis=0), label="greedy")
plt.plot(np.mean(100*optimal_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(100*optimal_softmax, axis=0), label="softmax")
plt.xlabel("Plays")
plt.ylabel("Optimal action (%)")
plt.ylim((0,100))
plt.legend()
plt.show()

**A:** with optimistic initialization, greedy action selection becomes the most efficient method: exploration is ensured by the fact that all actions be be executed at some point, as they can only be disappointing. The received rewards are always lower than the expectation at the beginning, so there is no need for additional exploration mechanisms. But it necessitates to know in advance what the maximal reward is...

## Reinforcement comparison

The problem with the previous **value-based** methods is that the Q-value estimates depend on the absolute magnitude of the rewards (by definition). The hyperparameters of the learning algorithms (learning rate, exploration, initial values) will therefore be very different depending on the scaling of the rewards (between 0 and 1, between -100 and 100, etc).

A way to get rid of this dependency is to introduce **preferences** $p_t(a)$ for each action instead of the estimated Q-values. Preferences should follow the Q-values: an action with a high Q-value should have a high Q-value and vice versa, but we do not care about its exact scaling.

In **reinforcement comparison**, we introduce a baseline $\tilde{r}_t$ which is the average received reward **regardless the action**, i.e. there is a single value for the whole problem. This average reward is simply updated after each action with a moving average of the received rewards:

$$\tilde{r}_{t+1} = \tilde{r}_{t} + \alpha \, (r_t - \tilde{r}_{t})$$

The average reward is used to update the preference for the action that was just executed:

$$p_{t+1}(a_t) = p_{t}(a_t) + \beta \, (r_t - \tilde{r}_{t})$$

If the action lead to more reward than usual, its preference should be increased (good surprise). If the action lead to less reward than usual, its preference should be decreased (bad surprise).

Action selection is simply a softmax over the preferences, without the temperature parameter (as we do not care about the scaling):

$$
    \pi (a) = \frac{\exp p_t(a)}{ \sum_b \exp p_t(b)}
$$ 

**Q:** Implement reinforcement comparison (with $\alpha=\beta=0.1$) and compare it to the other methods. 

In [ ]:
# Learning rate
alpha = 0.1

# Exploration
epsilon = 0.1
tau = 0.1
alpha_rc = 0.1
beta_rc = 0.1

# Setup
nb_trials = 200
nb_steps = 1000

# Store all received rewards and the optimality in a big matrix
rewards_greedy = np.zeros((nb_trials, nb_steps))
optimal_greedy = np.zeros((nb_trials, nb_steps))

rewards_egreedy = np.zeros((nb_trials, nb_steps))
optimal_egreedy = np.zeros((nb_trials, nb_steps))

rewards_softmax = np.zeros((nb_trials, nb_steps))
optimal_softmax = np.zeros((nb_trials, nb_steps))

rewards_rc = np.zeros((nb_trials, nb_steps))
optimal_rc = np.zeros((nb_trials, nb_steps))

# 200 runs
for trial in range(nb_trials):
    
    # True Q-values
    Q_star = rng.normal(0, 1, nb_actions)
    a_star = rng.choice(np.where(Q_star == Q_star.max())[0])
    
    # Estimates
    Q_greedy = np.zeros(nb_actions)
    Q_egreedy = np.zeros(nb_actions)
    Q_softmax = np.zeros(nb_actions)
    p_rc = np.zeros(nb_actions)
    mean_reward = 0.0
    
    # For 1000 plays
    for step in range(nb_steps):    
        
        # greedy
        a = rng.choice(np.where(Q_greedy == Q_greedy.max())[0])
        r = get_reward(Q_star, a)
        rewards_greedy[trial, step] = r
        if a == a_star:
            optimal_greedy[trial, step] = 1.0 
        Q_greedy[a] += alpha * (r - Q_greedy[a])
        
        # epsilon-greedy
        a = rng.choice(np.where(Q_egreedy == Q_egreedy.max())[0])
        if rng.random() < epsilon:
            a = rng.choice(actions[actions !=a])
        r = get_reward(Q_star, a)
        rewards_egreedy[trial, step] = r
        if a == a_star:
            optimal_egreedy[trial, step] = 1.0 
        Q_egreedy[a] += alpha * (r - Q_egreedy[a])
        
        # Softmax            
        logit = np.exp((Q_softmax - Q_softmax.max())/tau)
        proba_softmax = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_softmax) 
        r = get_reward(Q_star, a)
        rewards_softmax[trial, step] = r
        if a == a_star:
            optimal_softmax[trial, step] = 1.0            
        Q_softmax[a] += alpha * (r - Q_softmax[a])
        
        # Reinforcement comparison            
        logit = np.exp((p_rc - p_rc.max()))
        proba_rc = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_rc) 
        r = get_reward(Q_star, a)
        rewards_rc[trial, step] = r
        if a == a_star:
            optimal_rc[trial, step] = 1.0            
        p_rc[a] += beta_rc * (r - mean_reward)        
        mean_reward += alpha_rc * (r - mean_reward)
    
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.mean(rewards_greedy, axis=0), label="greedy")
plt.plot(np.mean(rewards_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(rewards_softmax, axis=0), label="softmax")
plt.plot(np.mean(rewards_rc, axis=0), label="reinforcement comparison")
plt.xlabel("Plays")
plt.ylabel("Mean Reward")
plt.legend()
plt.subplot(122)
plt.plot(np.mean(100*optimal_greedy, axis=0), label="greedy")
plt.plot(np.mean(100*optimal_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(100*optimal_softmax, axis=0), label="softmax")
plt.plot(np.mean(100*optimal_rc, axis=0), label="reinforcement comparison")
plt.xlabel("Plays")
plt.ylabel("Optimal action (%)")
plt.ylim((0,100))
plt.legend()
plt.show()

**A:** RC is slower at the beginning, but ends up being more optimal. We never estimate the Q-values, but we do not care about them, we only want to perform the correct actions. We also get rid of the temperature parameter.

**Q:** Compare all methods with optimistic initialization. The true Q-values come from $\mathcal{N}(10, 10)$, the estimated Q-values are initialized to 20 for greedy, $\epsilon$-greedy and softmax, and the average reward is initialized to 20 for RC (the preferences are initialized at 0).  

In [ ]:
# Learning rate
alpha = 0.1

# Exploration
epsilon = 0.1
tau = 0.1
alpha_rc = 0.1
beta_rc = 0.1

# Setup
nb_trials = 200
nb_steps = 1000

# Store all received rewards and the optimality in a big matrix
rewards_greedy = np.zeros((nb_trials, nb_steps))
optimal_greedy = np.zeros((nb_trials, nb_steps))

rewards_egreedy = np.zeros((nb_trials, nb_steps))
optimal_egreedy = np.zeros((nb_trials, nb_steps))

rewards_softmax = np.zeros((nb_trials, nb_steps))
optimal_softmax = np.zeros((nb_trials, nb_steps))

rewards_rc = np.zeros((nb_trials, nb_steps))
optimal_rc = np.zeros((nb_trials, nb_steps))

# 200 runs
for trial in range(nb_trials):
    
    # True Q-values
    Q_star = rng.normal(10, 10, nb_actions)
    a_star = rng.choice(np.where(Q_star == Q_star.max())[0])
    
    # Estimates
    Q_greedy = 20*np.ones(nb_actions)
    Q_egreedy = 20*np.ones(nb_actions)
    Q_softmax = 20*np.ones(nb_actions)
    p_rc = np.zeros(nb_actions)
    mean_reward = 20.0
    
    # For 1000 plays
    for step in range(nb_steps):    
        
        # greedy
        a = rng.choice(np.where(Q_greedy == Q_greedy.max())[0])
        r = get_reward(Q_star, a)
        rewards_greedy[trial, step] = r
        if a == a_star:
            optimal_greedy[trial, step] = 1.0 
        Q_greedy[a] += alpha * (r - Q_greedy[a])
        
        # epsilon-greedy
        a = rng.choice(np.where(Q_egreedy == Q_egreedy.max())[0])
        if rng.random() < epsilon:
            a = rng.choice(actions[actions !=a])
        r = get_reward(Q_star, a)
        rewards_egreedy[trial, step] = r
        if a == a_star:
            optimal_egreedy[trial, step] = 1.0 
        Q_egreedy[a] += alpha * (r - Q_egreedy[a])
        
        # Softmax            
        logit = np.exp((Q_softmax - Q_softmax.max())/tau)
        proba_softmax = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_softmax) 
        r = get_reward(Q_star, a)
        rewards_softmax[trial, step] = r
        if a == a_star:
            optimal_softmax[trial, step] = 1.0            
        Q_softmax[a] += alpha * (r - Q_softmax[a])
        
        # Reinforcement comparison            
        logit = np.exp((p_rc - p_rc.max()))
        proba_rc = logit / np.sum(logit)
        a = rng.choice(actions, p=proba_rc) 
        r = get_reward(Q_star, a)
        rewards_rc[trial, step] = r
        if a == a_star:
            optimal_rc[trial, step] = 1.0            
        p_rc[a] += beta_rc * (r - mean_reward)        
        mean_reward += alpha_rc * (r - mean_reward)
    
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.mean(rewards_greedy, axis=0), label="greedy")
# plt.plot(np.mean(rewards_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(rewards_softmax, axis=0), label="softmax")
plt.plot(np.mean(rewards_rc, axis=0), label="reinforcement comparison")
plt.xlabel("Plays")
plt.ylabel("Mean Reward")
plt.ylim((15,23))
plt.legend()
plt.subplot(122)
plt.plot(np.mean(100*optimal_greedy, axis=0), label="greedy")
# plt.plot(np.mean(100*optimal_egreedy, axis=0), label="epsilon-greedy")
plt.plot(np.mean(100*optimal_softmax, axis=0), label="softmax")
plt.plot(np.mean(100*optimal_rc, axis=0), label="reinforcement comparison")
plt.xlabel("Plays")
plt.ylabel("Optimal action (%)")
plt.ylim((50,100))
plt.legend()
plt.show()